# Setup

In [1]:
install_load <- function (package1, ...)  {   

   # convert arguments to vector
   packages <- c(package1, ...)

   # start loop to determine if each package is installed
   for(package in packages){

       # if package is installed locally, load
       if(package %in% rownames(installed.packages()))
          do.call('library', list(package))

       # if package is not installed locally, download, then load
       else {
          install.packages(package)
          do.call("library", list(package))
       }
   } 
}

# Section 9.2

In [2]:
install_load("tidyselect", "ggpubr", "parallel", "parallelMap", "randomForestSRC", "mlr", "tidyverse", "FSelector")

Loading required package: ggplot2

 randomForestSRC 2.9.3 
 
 Type rfsrc.news() to see new features, changes, and bug fixes. 
 

Loading required package: ParamHelpers
'mlr' is in maintenance mode since July 2019. Future development
efforts will go into its successor 'mlr3' (<https://mlr3.mlr-org.com>).

Attaching package: ‘mlr’

The following objects are masked from ‘package:randomForestSRC’:

    impute, subsample

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  2.1.3     ✔ purrr   0.3.3
✔ tidyr   1.1.0     ✔ dplyr   1.0.0
✔ readr   1.3.1     ✔ stringr 1.4.0
✔ tibble  2.1.3     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ purrr::partial() masks randomForestSRC::partial()


## Section 9.2.1

### Listing 9.1

In [3]:
data(Ozone, package = "mlbench")
ozoneTib <- as_tibble(Ozone)
names(ozoneTib) <- c("Month", "Date", "Day", "Ozone", "Press_height",
                     "Wind", "Humid", "Temp_Sand", "Temp_Monte",
                     "Inv_height", "Press_grad", "Inv_temp", "Visib")
head(ozoneTib)

Month,Date,Day,Ozone,Press_height,Wind,Humid,Temp_Sand,Temp_Monte,Inv_height,Press_grad,Inv_temp,Visib
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,4,3,5480,8,20,NA,NA,5000,-15,30.56,200
1,2,5,3,5660,6,NA,38,NA,NA,-14,NA,300
1,3,6,3,5710,4,28,40,NA,2693,-25,47.66,250
1,4,7,5,5700,3,37,45,NA,590,-24,55.04,100
1,5,1,5,5760,3,51,54,45.32,1450,25,57.02,60
1,6,2,6,5720,4,69,35,49.64,1568,15,53.78,60


In [4]:
# missing data
map_dbl(ozoneTib, ~sum(is.na(.)))

Month         Date          Day        Ozone Press_height         Wind 
           0            0            0            5           12            0 
       Humid    Temp_Sand   Temp_Monte   Inv_height   Press_grad     Inv_temp 
          15            2          139           15            1           14 
       Visib 
           0

### Listing 9.2

In [5]:
ozoneClean <- mutate_all(ozoneTib, as.numeric) %>%
    filter(is.na(Ozone) == FALSE)

head(ozoneClean)

Month,Date,Day,Ozone,Press_height,Wind,Humid,Temp_Sand,Temp_Monte,Inv_height,Press_grad,Inv_temp,Visib
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,4,3,5480,8,20,NA,NA,5000,-15,30.56,200
1,2,5,3,5660,6,NA,38,NA,NA,-14,NA,300
1,3,6,3,5710,4,28,40,NA,2693,-25,47.66,250
1,4,7,5,5700,3,37,45,NA,590,-24,55.04,100
1,5,1,5,5760,3,51,54,45.32,1450,25,57.02,60
1,6,2,6,5720,4,69,35,49.64,1568,15,53.78,60


### Listing 9.3

In [6]:
ozoneUntidy <- gather(ozoneClean, key = "Variable", value = "Value", -Ozone)

options(repr.plot.width=15, repr.plot.height=8)
ggplot(ozoneUntidy, aes(Value, Ozone)) +
    facet_wrap(~ Variable, scale = "free_x") + 
    geom_point() +
    geom_smooth() +
    geom_smooth(method = "lm", col = "red") + 
    theme_bw()

`geom_smooth()` using method = 'loess' and formula 'y ~ x'
Warning message:
“Removed 196 rows containing non-finite values (stat_smooth).”Warning message:
“Removed 196 rows containing non-finite values (stat_smooth).”Warning message:
“Removed 196 rows containing missing values (geom_point).”

## Section 9.2.2

### Listing 9.4

In [7]:
imputeMethod <- imputeLearner("regr.rpart")
ozoneImp <- impute(as.data.frame(ozoneClean), classes = list(numeric = imputeMethod))

### Listing 9.5

In [8]:
ozoneTask <- makeRegrTask(data = ozoneImp$data, target = "Ozone")
lin <- makeLearner("regr.lm")

### Listing 9.6

In [9]:
filterVals <- generateFilterValuesData(ozoneTask,
                                      method = "linear.correlation")
filterVals$data

name,type,filter,value
<chr>,<chr>,<fct>,<dbl>
Month,numeric,linear.correlation,0.05371420
Date,numeric,linear.correlation,0.08205094
Day,numeric,linear.correlation,0.04151444
Press_height,numeric,linear.correlation,0.58752354
Wind,numeric,linear.correlation,0.00468138
Humid,numeric,linear.correlation,0.45148094
Temp_Sand,numeric,linear.correlation,0.76977674
Temp_Monte,numeric,linear.correlation,0.74159034
Inv_height,numeric,linear.correlation,0.57563391


In [10]:
options(repr.plot.width=15, repr.plot.height=8)
plotFilterValues(filterVals) + theme_bw()

### Exercise 1

In [11]:
# use default method
filterValsRF <- generateFilterValuesData(ozoneTask, method = "randomForestSRC_importance")
filterValsRF$data

name,type,filter,value
<chr>,<chr>,<fct>,<dbl>
Month,numeric,randomForestSRC_importance,1.34084571
Date,numeric,randomForestSRC_importance,-0.05838480
Day,numeric,randomForestSRC_importance,-0.05883131
Press_height,numeric,randomForestSRC_importance,0.67575815
Wind,numeric,randomForestSRC_importance,-0.02202729
Humid,numeric,randomForestSRC_importance,2.40736490
Temp_Sand,numeric,randomForestSRC_importance,9.06512414
Temp_Monte,numeric,randomForestSRC_importance,7.55080427
Inv_height,numeric,randomForestSRC_importance,2.04242237


In [12]:
# Press_grad, Humid ranked higher in default filter 
plotFilterValues(filterValsRF) + theme_bw()

### Listing 9.7

In [28]:
# choose the best 6
ozoneFiltTaskBest6 <- filterFeatures(ozoneTask, 
                               fval = filterVals, abs = 6)
ozoneFiltFeatBest6 <- generateFilterValuesData(ozoneFiltTaskBest6, method = "linear.correlation")
p1 <- plotFilterValues(ozoneFiltFeatBest6) + theme_bw()

# choose the best 25%
ozoneFiltTaskTopQuartile <- filterFeatures(ozoneTask,
                               fval = filterVals, per = 0.25)
ozoneFiltFeatTopQuartile <- generateFilterValuesData(ozoneFiltTaskTopQuartile, method = "linear.correlation")
p2 <- plotFilterValues(ozoneFiltFeatTopQuartile) + theme_bw()


# choose above 20% correlation coefficient
ozoneFiltTaskTop20Percent <- filterFeatures(ozoneTask,
                               fval = filterVals, threshold = 0.2)
ozoneFiltFeatTop20Percent <- generateFilterValuesData(ozoneFiltTaskTop20Percent, method = "linear.correlation")
p3 <- plotFilterValues(ozoneFiltFeatTop20Percent) + theme_bw()

figure <- ggarrange(p1, p2, p3,
                    labels = c("A", "B", "C"),
                    ncol = 3, nrow = 1)
figure


### Listing 9.8

In [29]:
filterWrapper <- makeFilterWrapper(learner = lin, 
                                  fw.method = "linear.correlation")

### Listing 9.9

In [30]:
lmParamSpace <- makeParamSet(
    makeIntegerParam("fw.abs", lower = 1, upper = 12))

gridSearch <- makeTuneControlGrid()

kFold <- makeResampleDesc("CV", iters = 10)

tunedFeats <- tuneParams(filterWrapper, task = ozoneTask,
                        resampling = kFold,
                        par.set = lmParamSpace, 
                        control = gridSearch)

tunedFeats

[Tune] Started tuning learner regr.lm.filtered for parameter set:
          Type len Def  Constr Req Tunable Trafo
fw.abs integer   -   - 1 to 12   -    TRUE     -
With control class: TuneControlGrid
Imputation value: Inf
[Tune-x] 1: fw.abs=1
[Tune-y] 1: mse.test.mean=25.6351083; time: 0.0 min
[Tune-x] 2: fw.abs=2
[Tune-y] 2: mse.test.mean=25.0896089; time: 0.0 min
[Tune-x] 3: fw.abs=3
[Tune-y] 3: mse.test.mean=24.8061331; time: 0.0 min
[Tune-x] 4: fw.abs=5
[Tune-y] 4: mse.test.mean=22.5118901; time: 0.0 min
[Tune-x] 5: fw.abs=6
[Tune-y] 5: mse.test.mean=20.9365187; time: 0.0 min
[Tune-x] 6: fw.abs=7
[Tune-y] 6: mse.test.mean=20.9978692; time: 0.0 min
[Tune-x] 7: fw.abs=8
[Tune-y] 7: mse.test.mean=21.0381248; time: 0.0 min
[Tune-x] 8: fw.abs=10
[Tune-y] 8: mse.test.mean=20.8182448; time: 0.0 min
[Tune-x] 9: fw.abs=11
[Tune-y] 9: mse.test.mean=20.6172512; time: 0.0 min
[Tune-x] 10: fw.abs=12
[Tune-y] 10: mse.test.mean=20.6549711; time: 0.0 min
[Tune] Result: fw.abs=11 : mse.test.mean=20

Tune result:
Op. pars: fw.abs=11
mse.test.mean=20.6172512

### Exercise 2

In [31]:
# repeat 9.8 with defaults
filterWrapperWithDefault <- makeFilterWrapper(learner = lin)

# repeat 9.9 with new wrapper
lmParamSpace2 <- makeParamSet(
    makeIntegerParam("fw.abs", lower = 1, upper = 12))

gridSearch2 <- makeTuneControlGrid()

kFold2 <- makeResampleDesc("CV", iters = 10)

tunedFeats2 <- tuneParams(filterWrapperWithDefault, task = ozoneTask,
                        resampling = kFold2,
                        par.set = lmParamSpace2, 
                        control = gridSearch2)

tunedFeats2

[Tune] Started tuning learner regr.lm.filtered for parameter set:
          Type len Def  Constr Req Tunable Trafo
fw.abs integer   -   - 1 to 12   -    TRUE     -
With control class: TuneControlGrid
Imputation value: Inf
[Tune-x] 1: fw.abs=1
[Tune-y] 1: mse.test.mean=25.7502419; time: 0.6 min
[Tune-x] 2: fw.abs=2
[Tune-y] 2: mse.test.mean=25.3994388; time: 0.7 min
[Tune-x] 3: fw.abs=3
[Tune-y] 3: mse.test.mean=25.0797962; time: 0.6 min
[Tune-x] 4: fw.abs=5
[Tune-y] 4: mse.test.mean=22.4604788; time: 0.6 min
[Tune-x] 5: fw.abs=6
[Tune-y] 5: mse.test.mean=21.3411689; time: 0.6 min
[Tune-x] 6: fw.abs=7
[Tune-y] 6: mse.test.mean=20.8166468; time: 0.6 min
[Tune-x] 7: fw.abs=8
[Tune-y] 7: mse.test.mean=20.5739922; time: 0.6 min
[Tune-x] 8: fw.abs=10
[Tune-y] 8: mse.test.mean=20.5299589; time: 0.6 min
[Tune-x] 9: fw.abs=11
[Tune-y] 9: mse.test.mean=20.7952459; time: 0.6 min
[Tune-x] 10: fw.abs=12
[Tune-y] 10: mse.test.mean=20.9109025; time: 0.6 min
[Tune] Result: fw.abs=10 : mse.test.mean=20

Tune result:
Op. pars: fw.abs=10
mse.test.mean=20.5299589

In [38]:
# Filter wrapper method linear.correlation was much faster (0 min vs 0.6 min for each parameter)
# The default method chooses fewer features (10 vs 11)

### Listing 9.10

In [39]:
filteredTask <- filterFeatures(ozoneTask, fval = filterVals,
                              abs = unlist(tunedFeats$x))
filteredModel <- train(lin, filteredTask)

### Listing 9.11

In [40]:
featSelControl <- makeFeatSelControlSequential(method = "sfbs")

selFeats <- selectFeatures(learner = lin, task = ozoneTask,
                          resampling = kFold, control = featSelControl)
selFeats

[FeatSel] Started selecting features for learner 'regr.lm'
With control class: FeatSelControlSequential
Imputation value: Inf
[FeatSel-x] 1: 111111111111 (12 bits)
[FeatSel-y] 1: mse.test.mean=20.6787711; time: 0.0 min
[FeatSel-x] 2: 011111111111 (11 bits)
[FeatSel-y] 2: mse.test.mean=21.3566316; time: 0.0 min
[FeatSel-x] 2: 101111111111 (11 bits)
[FeatSel-y] 2: mse.test.mean=20.5506336; time: 0.0 min
[FeatSel-x] 2: 110111111111 (11 bits)
[FeatSel-y] 2: mse.test.mean=20.5933256; time: 0.0 min
[FeatSel-x] 2: 111011111111 (11 bits)
[FeatSel-y] 2: mse.test.mean=20.7625914; time: 0.0 min
[FeatSel-x] 2: 111101111111 (11 bits)
[FeatSel-y] 2: mse.test.mean=20.6558331; time: 0.0 min
[FeatSel-x] 2: 111110111111 (11 bits)
[FeatSel-y] 2: mse.test.mean=21.4963548; time: 0.0 min
[FeatSel-x] 2: 111111011111 (11 bits)
[FeatSel-y] 2: mse.test.mean=21.7097689; time: 0.0 min
[FeatSel-x] 2: 111111101111 (11 bits)
[FeatSel-y] 2: mse.test.mean=21.2273729; time: 0.0 min
[FeatSel-x] 2: 111111110111 (11 bits)

FeatSel result:
Features (6): Month, Press_height, Humid, Temp_Sand, Temp_Mon...
mse.test.mean=20.1451656

### Listing 9.12

In [41]:
ozoneSelFeat <- ozoneImp$data[, c("Ozone", selFeats$x)]

ozoneSelFeatTask <- makeRegrTask(data = ozoneSelFeat, target = "Ozone")

wrapperModel <- train(lin, ozoneSelFeatTask)

## Section 9.2.4

### Listing 9.13

In [43]:
imputeMethod <- imputeLearner("regr.rpart")

imputeWrapper <- makeImputeWrapper(lin, 
                                 classes = list(numeric = imputeMethod))

featSelWrapper <- makeFeatSelWrapper(learner = imputeWrapper,
                                    resampling = kFold,
                                    control = featSelControl)

### Listing 9.14

In [ ]:
ozoneTaskWithNAs <- makeRegrTask(data = ozoneClean, target = "Ozone")

kFold3 <- makeResampleDesc("CV", iters = 3)

parallelStartSocket(cpus = detectCores())

lmCV <- resample(featSelWrapper, ozoneTaskWithNAs, resampling = kFold3)

parallelStop()

lmCV